# Data Monitoring Automation

This notebook documents the code and progress as I work my way towards setting up an automated process to monitor the performance of the fraud model(s), across partners and products. 

In [1]:
# Library Imports

import pandas as pd
import numpy as np
import trellis
import os
from avant_python_utils.email import send_email
from datalaketools.connectors.presto_db import PrestoDB
presto = PrestoDB()
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, f1_score,recall_score,precision_score, average_precision_score
from datetime import date, timedelta, datetime

## Constants with Column Names

Lots of columns that are used in multiple functions throughout this document. I'll define those columns here so I won't have to change them in multiple places later if there's a change

In [19]:
# TODO - Define these as global variables (store them in a config.py file)

SCORE_COL = 'score_5'
YPRED_COL = 'prediction'
YTRUE_COL = 'suspected_fraud'
TIME_COL = 'loan_processing_start_time'
#TIME_COL = 'created_at'
AMOUNT_COL = 'loan_amount'
THRESHOLD = 0.05
LOAN_WINDOW = 'week'
MODEL_START_DATE = '2018-09-15'





## Base Data Creation

In [20]:
#trellis.start()
# fraud = trellis.connect('us_fraud_follower')
#parent_dir_path = os.path.dirname(os.path.abspath(__file__)) - REMOVE COMMENT IN PYTHON SCRIPT
parent_dir_path = os.getcwd()
subject = 'Avant Model Monitor Weekly Report (Data Only)'
credentials = {'username': trellis.keys('automate_email')['email'], 'password': trellis.keys('automate_email')['pw']}


In [4]:
# TD prod connection
td_connector = trellis.connect('us_fraud_follower')

In [21]:
df_raw = presto.execute_df('''
SELECT
  l.id as loan_id
, l.loan_processing_start_time
, date_trunc('{LOAN_WINDOW}', l.loan_processing_start_time) as entered_lp_week
, l.status
, case when l.status in ('current','late','paid_off','charged_off') then 1 else 0 end as issued
, case when c.high_confidence_fraud_indicator=true or cfl.id is not null then 1 else 0 end as high_confidence_fraud_indicator
, case when cfr.customer_id is not null then 1 else 0 end as suspected_fraud 
--, cfrt.name as fraud_reason
, cast(fd.score_5_old as double) as score_5_old
, cast(fd.score_5_new as double) as score_5_new
, coalesce(cast(fd.score_5_old as double), cast(fd.score_5_new as double)) as score_5
, l.state
, l.payment_method
, l.loan_amount
, ca.product_type
, vrdt.risk_summary_identity_high
, vrdt.risk_summary_identity_medium
, vrdt.risk_summary_identity_low
FROM avant.dw.customer_applications ca
LEFT JOIN avant.dw.loans l on l.customer_application_id = ca.id
JOIN avant.dw.customers c
  ON c.id = l.customer_id
  
  -- getting dependent variable
  
LEFT JOIN (
select customer_id 
from avant.avant_basic.customer_fraud_reasons cfr 
group by 1
) cfr on c.id = cfr.customer_id
  
 -- LEFT JOIN avant.avant_basic.customer_fraud_reason_types cfrt on cfr.customer_fraud_reason_type_id = cfrt.id
  
  -- getting fraud scores
LEFT JOIN (
  SELECT
    l.id as loan_id
  , json_extract_scalar(fd.model_scores, '$["fraud/en-US/4.1.0"]["score"]') as score_4
  , json_extract_scalar(fd.model_scores, '$["fraud/en-US/5.0.0"]["score"]') as score_5_old
  , json_extract_scalar(fd.model_scores, '$["fraud/en-US/5.0.0/avant"]["score"]') as score_5_new
  
  , fd.id as fraud_decision_id
  , row_number() over (partition by l.id order by fd.created_at desc) as row_num
  FROM avant.dw.loans l
  JOIN avant.avant_basic.fraud_decisions fd
    ON fd.customer_application_id = l.customer_application_id
    AND fd.created_at AT TIME ZONE 'America/Chicago' >= l.loan_processing_start_time
WHERE l.loan_processing_start_time > date '{START_DATE}'
) fd 
  ON fd.loan_id = l.id 
  AND fd.row_num=1
  -- getting fraud indicator
LEFT JOIN avant.avant_basic.confirmed_fraud_logs cfl 
  ON cfl.customer_id = c.id
  
    -- filtering for valid loans to evaluate performance on
  -- JOIN avant.dw.loan_performance_by_installment lp 
  -- ON lp.loan_id = l.id 
  -- AND lp.installment_number = 1
  -- AND lp.installment_date <= date_add('day', -64, current_timestamp)

  
  -- adding identity tier a loan was assigned to and fraud_review flag
  LEFT JOIN avant.dw_temp_newver.verifications_risks_decisions_test vrdt
  on ca.id = vrdt.customer_application_id and vrdt.row_num_recent = 1
  
  
WHERE l.loan_processing_start_time > date '{START_DATE}'
'''.format(LOAN_WINDOW = LOAN_WINDOW, START_DATE = MODEL_START_DATE))


In [22]:
#Remove records with no fraud score
df = df_raw[df_raw.score_5.notnull()]

In [23]:
max(df[TIME_COL])

'2020-10-07 21:46:17.937'

In [24]:
df

,loan_id,loan_processing_start_time,entered_lp_week,status,issued,high_confidence_fraud_indicator,suspected_fraud,score_5_old,score_5_new,score_5,state,payment_method,loan_amount,product_type,risk_summary_identity_high,risk_summary_identity_medium,risk_summary_identity_low
0,3606862,2019-06-24 13:31:17.246,2019-06-24 00:00:00.000,current,1,0,0,0.012123,NaN,0.012123,CA,ach,4869.07,refinance,False,False,True
1,3919825,2020-03-15 19:13:18.697,2020-03-09 00:00:00.000,rejected,0,0,0,0.105871,NaN,0.105871,NC,ach,8000.00,installment,False,True,False
2,3411208,2019-02-22 17:26:47.037,2019-02-18 00:00:00.000,current,1,0,0,0.005874,NaN,0.005874,FL,ach,6400.00,installment,False,False,True
3,4007677,2020-09-11 11:38:45.820,2020-09-07 00:00:00.000,current,1,0,0,NaN,0.002764,0.002764,CT,ach,3502.57,refinance,False,False,True
4,3203849,2018-10-17 14:54:29.681,2018-10-15 00:00:00.000,cancelled,0,0,0,0.007981,NaN,0.007981,MA,ach,6100.00,installment,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502655,3165512,2018-09-25 11:25:24.542,2018-09-24 00:00:00.000,paid_off,1,0,1,0.003966,NaN,0.003966,LA,ach,2700.00,installment,False,False,True
502656,3227565,2018-10-31 14:20:25.276,2018-10-29 00:00:00.000,paid_off,1,0,0,0.002969,NaN,0.002969,NE,ach,5300.00,installment,False,False,True
502657,3721309,2019-09-05 20:45:46.177,2019-09-02 00:00:00.000,current,1,0,0,0.022613,NaN,0.022613,AZ,ach,10000.00,installment,False,False,True
502658,3829579,2019-12-02 14:15:53.616,2019-12-02 00:00:00.000,rejected,0,1,1,0.009197,NaN,0.009197,TX,ach,4000.00,installment,False,False,True


In [25]:
df[YPRED_COL] = np.where(df[SCORE_COL] > THRESHOLD, 1, 0)
#df['prediction'] = [1 if x > 0.05 else 0 for x in df['score_5']]
#df['prediction'] = list(np.where(df['score_5'] > 0.05, 1, 0)) 
# All three solutions raise the settingwithcopywarining

/home/jovyan/.conda-envs/gkrishna_env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## Model Evaluation Pipeline

In [26]:
def weeklyEvaluator(dframe, ytrue = YTRUE_COL, ypred = YPRED_COL, scores = SCORE_COL):
    true_positives = dframe[ytrue] * dframe[ypred]
    false_positives = (1-dframe[ytrue]) * dframe[ypred]
    false_negatives =  dframe[ytrue] * (1-dframe[ypred])
    
    #calculating multiple metrics
    precision = precision_score(y_true = dframe[ytrue], y_pred = dframe[ypred], pos_label = 1)
    recall = recall_score(y_true = dframe[ytrue], y_pred = dframe[ypred], pos_label = 1)
    f1score = f1_score(y_true = dframe[ytrue], y_pred = dframe[ypred], pos_label = 1)
    auc_pr = average_precision_score(y_true = dframe[ytrue], y_score = dframe[scores], pos_label=1)
    auc_roc = roc_auc_score(y_true = dframe[ytrue], y_score = dframe[scores])
    fraud_rate = dframe[ytrue].sum()/len(dframe.index)
    avg_score = dframe[scores].sum()/len(dframe.index)
    
    
    
    return pd.Series({'precision': precision, 'recall': recall, 'f1score': f1score, 'auc_pr':auc_pr, 'auc_roc':auc_roc,
                     'fraud_rate': fraud_rate, 'avg_score': avg_score})

Figuring out the date from which we have both fraud and not fraud in the same week, so that we can group by that week. 

In [27]:
#unique fraud values per week

test = pd.DataFrame(df.groupby('entered_lp_week', as_index = False)['suspected_fraud'].nunique())

In [28]:
test.query('suspected_fraud == 1')

,entered_lp_week,suspected_fraud


In [29]:
byWeek_stats = df.groupby('entered_lp_week', as_index = False).apply(weeklyEvaluator)

## Connecting to Google Sheets

In [10]:
#Importing the module
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

In [49]:
#The scope is always look like this so we did not need to change anything
scope = [
   'https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
#Name of our Service Account Key
google_key_file = 'service_key.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(google_key_file, scope)
gc = gspread.authorize(credentials)

In [54]:
#This is the Worksheet ID
wb = gc.open_by_key('14ROlpuOP9IkixM5-nn1Pc0ux6kWgmj7c62NzdDl-5hU')
#This is the sheet name
wks_name = 'Raw Data'
ws = wb.worksheet(wks_name)

#clear the existing data in worksheet
ws.clear()

#update new data to worksheet (first list out columns, and then add values for each column)
ws.update([byWeek_stats.columns.values.tolist()] + byWeek_stats.values.tolist())

{'spreadsheetId': '14ROlpuOP9IkixM5-nn1Pc0ux6kWgmj7c62NzdDl-5hU',
 'updatedRange': "'Raw Data'!A1:H104",
 'updatedRows': 104,
 'updatedColumns': 8,
 'updatedCells': 832}

In [55]:
ws.format("A2:A1000", { "numberFormat": { "type": ('DATE') }})

{'spreadsheetId': '14ROlpuOP9IkixM5-nn1Pc0ux6kWgmj7c62NzdDl-5hU',
 'replies': [{}]}

## Filling in Table Values

For each metric, I need the following 

- Value since the date model was trained
- Compare last 30 days to previous 30 days (with scope for excluding the last 2 months)
- Absolute difference between value when model was trained, and current value

In [48]:
datetime.strptime(a, "%Y-%m-%d") + timedelta(days = 30)

datetime.datetime(2018, 10, 15, 0, 0)

In [27]:
test = df.query('loan_processing_start_time > "2019-09-15" & loan_processing_start_time < "2019-10-30"')

In [52]:
today_date

'2020-10-05'

In [30]:
today_date = date.today().strftime("%Y-%m-%d")
prev30_date = (date.today() - timedelta(days = 30)).strftime("%Y-%m-%d")
prev60_date = (date.today() - timedelta(days = 60)).strftime("%Y-%m-%d")
#a = datetime.strptime(MODEL_START_DATE, "%Y-%m-%d")
#b = (a + timedelta(days = 30)).strftime("%Y-%m-%d")

#creating different datasets for the different time periods

#dataset 1 - 30 days after model was trained


#test = df.query('{0} > @MODEL_START_DATE & {0} < @b'.format(TIME_COL))
data_last30 = df.query('loan_processing_start_time > @prev30_date & loan_processing_start_time < @today_date')
#data_prev30 = dframe.query('@timecol > @prev60_date & @timecol < @prev30_date')   

In [31]:
data_last30

,loan_id,loan_processing_start_time,entered_lp_week,status,issued,high_confidence_fraud_indicator,suspected_fraud,score_5_old,score_5_new,score_5,state,payment_method,loan_amount,product_type,risk_summary_identity_high,risk_summary_identity_medium,risk_summary_identity_low,prediction
3,4007677,2020-09-11 11:38:45.820,2020-09-07 00:00:00.000,current,1,0,0,NaN,0.002764,0.002764,CT,ach,3502.57,refinance,False,False,True,0
42,4011055,2020-09-16 16:36:22.470,2020-09-14 00:00:00.000,current,1,0,0,NaN,0.003041,0.003041,AZ,ach,3000.00,installment,False,False,True,0
294,4024656,2020-10-07 08:41:36.366,2020-10-05 00:00:00.000,current,1,0,0,NaN,0.002811,0.002811,NM,ach,5300.00,installment,False,False,True,0
380,4024871,2020-10-07 13:32:55.339,2020-10-05 00:00:00.000,applied,0,0,0,NaN,0.002819,0.002819,MI,ach,10500.00,installment,False,False,True,0
465,4009995,2020-09-15 10:30:41.743,2020-09-14 00:00:00.000,current,1,0,0,NaN,0.030648,0.030648,CA,ach,5886.98,refinance,False,False,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502539,4021919,2020-10-02 13:30:29.205,2020-09-28 00:00:00.000,current,1,0,0,NaN,0.004403,0.004403,PA,ach,3200.00,installment,False,False,True,0
502579,4020874,2020-10-01 02:31:17.350,2020-09-28 00:00:00.000,rejected,0,0,0,NaN,0.372705,0.372705,NH,ach,5300.00,installment,False,True,False,1
502617,4023197,2020-10-05 09:53:07.951,2020-10-05 00:00:00.000,current,1,0,0,NaN,0.004795,0.004795,VA,ach,2701.66,refinance,False,False,True,0
502622,4015983,2020-09-23 21:27:49.428,2020-09-21 00:00:00.000,current,1,0,0,NaN,0.007211,0.007211,MO,ach,10000.00,installment,False,False,True,0


In [32]:
average_precision_score(y_true = data_last30[YTRUE_COL], y_score = data_last30[SCORE_COL], pos_label = 1)

0.332906862189837

In [41]:
test = values_for_cells(df)

In [40]:
#function to return a metric for a specified time period

def values_for_cells(dframe, ytrue = YTRUE_COL, ypred = YPRED_COL, scores = SCORE_COL, timecol = TIME_COL, amount = AMOUNT_COL):
   
    #Setting up variables with different date values
    today_date = date.today().strftime("%Y-%m-%d")
    prev30_date = (date.today() - timedelta(days = 30)).strftime("%Y-%m-%d")
    prev60_date = (date.today() - timedelta(days = 60)).strftime("%Y-%m-%d")
    modeltrain_date_start = datetime.strptime(MODEL_START_DATE, "%Y-%m-%d")
    modeltrain_date_end = (modeltrain_date_start + timedelta(days = 30)).strftime("%Y-%m-%d")
    
    #creating different datasets for the different time periods
    
    #dataset 1 - 30 days after model was trained

    
    data_first30 = dframe.query('{0} > @MODEL_START_DATE & {0} < @modeltrain_date_end'.format(TIME_COL))
    data_last30 = dframe.query('{0} > @prev30_date & {0} < @today_date'.format(TIME_COL))
    data_prev30 = dframe.query('{0} > @prev60_date & {0} < @prev30_date'.format(TIME_COL))   

    #PRECISION
    precision_current = precision_score(y_true = data_last30[ytrue], y_pred = data_last30[ypred], pos_label = 1)
    precision_initial = precision_score(y_true = data_first30[ytrue], y_pred = data_first30[ypred], pos_label = 1)
    precision_prev30 = precision_score(y_true = data_prev30[ytrue], y_pred = data_prev30[ypred], pos_label = 1)

    #recall values
    recall_current = recall_score(y_true = data_last30[ytrue], y_pred = data_last30[ypred], pos_label = 1)
    recall_initial = recall_score(y_true = data_first30[ytrue], y_pred = data_first30[ypred], pos_label = 1)
    recall_prev30 = recall_score(y_true = data_prev30[ytrue], y_pred = data_prev30[ypred], pos_label = 1) 

    #F1 score
    f1_current = f1_score(y_true = data_last30[ytrue], y_pred = data_last30[ypred], pos_label = 1)
    f1_initial = f1_score(y_true = data_first30[ytrue], y_pred = data_first30[ypred], pos_label = 1)
    f1_prev30 = f1_score(y_true = data_prev30[ytrue], y_pred = data_prev30[ypred], pos_label = 1) 

    #auc pr
    aucpr_current = average_precision_score(y_true = data_last30[ytrue], y_score = data_last30[scores], pos_label = 1)
    aucpr_initial = average_precision_score(y_true = data_first30[ytrue], y_score = data_first30[scores], pos_label = 1)
    aucpr_prev30 = average_precision_score(y_true = data_prev30[ytrue], y_score = data_prev30[scores], pos_label = 1) 

    #auc roc
    aucroc_current = roc_auc_score(y_true = data_last30[ytrue], y_score = data_last30[scores])
    aucroc_initial = roc_auc_score(y_true = data_first30[ytrue], y_score = data_first30[scores])
    aucroc_prev30 = roc_auc_score(y_true = data_prev30[ytrue], y_score = data_prev30[scores]) 

    #TODO - Confirm fraud rate definition
    #fraud rate
    fraudrate_current = data_last30[ytrue].sum()/len(data_last30.index)
    fraudrate_initial = data_first30[ytrue].sum()/len(data_first30.index)
    fraudrate_prev30 = data_prev30[ytrue].sum()/len(data_prev30.index)
    
    #avg score
    avgscore_current = data_last30[scores].sum()/len(data_last30.index)
    avgscore_initial = data_first30[scores].sum()/len(data_first30.index)
    avgscore_prev30 = data_prev30[scores].sum()/len(data_prev30.index)


    #TODO - Confirm fraud missed definition
    #fraud rate with dollar values
    fraudrate_dollar_current = (data_last30[amount]*data_last30[YTRUE_COL]).sum()/data_last30[amount].sum()
    fraudrate_dollar_initial = (data_first30[amount]*data_first30[YTRUE_COL]).sum()/data_first30[amount].sum()
    fraudrate_dollar_prev30 = (data_prev30[amount]*data_prev30[YTRUE_COL]).sum()/data_prev30[amount].sum()

    #$ value of fraud missed
    fraudmissed_dollar_current = data_last30[scores].sum()/len(data_last30.index)
    fraudmissed_dollar_initial = data_first30[scores].sum()/len(data_first30.index)
    fraudmissed_dollar_prev30 = data_prev30[scores].sum()/len(data_prev30.index)

    output = {"metric": ['precision', 'recall','f1score', 'auc_pr', 'auc_roc', 'fraudrate', 'avg_score', 'fraudrate_dollar', 'fraudmissed_dollar'],
             "current_values":[precision_current, recall_current, f1_current, aucpr_current, aucroc_current, fraudrate_current, avgscore_current, fraudrate_dollar_current, fraudmissed_dollar_current],
             "initial_values":[precision_initial, recall_initial, f1_initial, aucpr_initial, aucroc_initial, fraudrate_initial, avgscore_initial, fraudrate_dollar_initial, fraudmissed_dollar_initial],
             "prev30_values":[precision_prev30, recall_prev30, f1_prev30, aucpr_prev30, aucroc_prev30, fraudrate_prev30, avgscore_prev30, fraudrate_dollar_prev30, fraudmissed_dollar_prev30]}    
        
    return output
    
        

In [43]:
pd.DataFrame.from_dict(test)

,metric,current_values,initial_values,prev30_values
0,precision,0.146799,0.123867,0.206675
1,recall,0.730769,0.465028,0.728507
2,f1score,0.244485,0.195626,0.322000
3,auc_pr,0.332907,0.187434,0.444112
4,auc_roc,0.895721,0.773095,0.901794
5,fraudrate,0.016518,0.020078,0.022359
6,avg_score,0.032695,0.025807,0.034249
7,fraudrate_dollar,0.020325,0.022331,0.026507
8,fraudmissed_dollar,0.032695,0.025807,0.034249
